# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherdf=pd.read_csv("output_data/cities.csv")
weatherdf.head()

,Unnamed: 0,City_ID,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,3887127,Iquique,0,CL,1603843991,68,-20.22,-70.14,64.40,13.87
1,1,934322,Mahébourg,3,MU,1603843991,80,-20.41,57.70,70.00,1.99
2,2,1637001,Biak,99,ID,1603843775,67,-0.91,122.88,82.62,3.74
3,3,360630,Cairo,20,EG,1603843951,73,30.06,31.25,71.60,6.93
4,4,3833367,Ushuaia,40,AR,1603843840,39,-54.80,-68.30,46.40,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations= weatherdf[['Lat','Lng']]
humidity = weatherdf['Humidity']


In [4]:
fig=gmaps.figure(center=(23,0),zoom_level=1)
humidity_heatmap=gmaps.heatmap_layer(locations,humidity,dissipating=False, max_intensity=100,point_radius=5)
fig.add_layer(humidity_heatmap)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
NarrowDF=weatherdf.loc[weatherdf["Cloudiness"]==0]
NarrowDF=NarrowDF.loc[NarrowDF["Wind Speed"]<10]
NarrowDF=NarrowDF.loc[(NarrowDF["Max Temp"]<80) & (NarrowDF["Max Temp"]>70)]
NarrowDF                      

,Unnamed: 0,City_ID,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
33,33,3446619,Terenos,0,BR,1603843999,73,-20.44,-54.86,71.60,1.12
40,40,1307741,Myitkyina,0,MM,1603844000,83,25.38,97.40,70.02,2.59
68,68,935214,Saint-Pierre,0,RE,1603844007,64,-21.34,55.48,71.60,2.24
111,111,935215,Saint-Philippe,0,RE,1603844017,64,-21.36,55.77,71.60,2.24
129,129,974933,Mogwase,0,ZA,1603844021,37,-25.18,27.27,73.20,4.29
133,133,2455290,Kidal,0,ML,1603844022,16,18.44,1.41,78.24,7.00
138,138,2439376,Tahoua,0,NE,1603844023,29,14.89,5.27,77.00,2.24
155,155,2447513,Arlit,0,NE,1603844027,15,18.74,7.39,79.00,7.78
168,168,139817,Bushehr,0,IR,1603844030,55,28.97,50.84,74.93,2.68
226,226,305681,Kumluca,0,TR,1603844044,45,36.37,30.29,71.38,3.96


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
NarrowDF["Hotel Name" ]=""

In [7]:
hotel_df=NarrowDF

In [8]:
hotel_df.head(8)

,Unnamed: 0,City_ID,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
33,33,3446619,Terenos,0,BR,1603843999,73,-20.44,-54.86,71.60,1.12,
40,40,1307741,Myitkyina,0,MM,1603844000,83,25.38,97.40,70.02,2.59,
68,68,935214,Saint-Pierre,0,RE,1603844007,64,-21.34,55.48,71.60,2.24,
111,111,935215,Saint-Philippe,0,RE,1603844017,64,-21.36,55.77,71.60,2.24,
129,129,974933,Mogwase,0,ZA,1603844021,37,-25.18,27.27,73.20,4.29,
133,133,2455290,Kidal,0,ML,1603844022,16,18.44,1.41,78.24,7.00,
138,138,2439376,Tahoua,0,NE,1603844023,29,14.89,5.27,77.00,2.24,
155,155,2447513,Arlit,0,NE,1603844027,15,18.74,7.39,79.00,7.78,


In [9]:
hotel_df.shape

(19, 12)

In [10]:
# parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Get hotels with latitude and longitude
for index, row in hotel_df.iterrows():
    
# get lat and lng from DF
    lat = row["Lat"]
    lng = row["Lng"]

# Iteration with locations
    params["location"] = f"{lat},{lng}"
    
# Search "Hotel Name" and request information from URL
    initial_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels_lat_lng= requests.get(initial_url, params=params)

#convert to json
    hotels_lat_lng = hotels_lat_lng.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_lat_lng["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotels_lat_lng["results"][0]["vicinity"]

######recomended to validate
        print(f"Closest hotel is {hotels_lat_lng['results'][0]['name']}.")
        
    except (KeyError, IndexError):

        print("Missing field/result... skipping.")
hotel_df.head(8)

Closest hotel is QUINTAS IN CASA.
Closest hotel is Shwe Phyu Hotel.
Closest hotel is Lindsey Hôtel.
Closest hotel is Chambres d'hôte "La Trinité".
Closest hotel is De Skaap Accommodation & Event Venue.
Missing field/result... skipping.
Closest hotel is Bungalows Guesthouse.
Closest hotel is Hôtel Telwa Bungalow.
Closest hotel is Yas Hotel.
Closest hotel is TurkuazKöy.
Closest hotel is La Caz Oceane.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
Closest hotel is Hotel Vivendas Rio Claro by Atlantica.
Closest hotel is Madarounfa centre ville.
Closest hotel is Desert Palms Alice Springs.
Closest hotel is Beach Rotana.
Closest hotel is Sur Plaza Hotel.
Closest hotel is "Plantation Bed and Breakfast.
Missing field/result... skipping.


,Unnamed: 0,City_ID,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Address
33,33,3446619,Terenos,0,BR,1603843999,73,-20.44,-54.86,71.60,1.12,QUINTAS IN CASA,"Estrada da ponte do Grego - Rural, Terenos"
40,40,1307741,Myitkyina,0,MM,1603844000,83,25.38,97.40,70.02,2.59,Shwe Phyu Hotel,MaMa 41-42-43 Corner of PyiTaungSu Road & ThaK...
68,68,935214,Saint-Pierre,0,RE,1603844007,64,-21.34,55.48,71.60,2.24,Lindsey Hôtel,21 Rue Francois Isautier
111,111,935215,Saint-Philippe,0,RE,1603844017,64,-21.36,55.77,71.60,2.24,"Chambres d'hôte ""La Trinité""",12 Rue De La Pompe
129,129,974933,Mogwase,0,ZA,1603844021,37,-25.18,27.27,73.20,4.29,De Skaap Accommodation & Event Venue,"Farm, Sandfontein"
133,133,2455290,Kidal,0,ML,1603844022,16,18.44,1.41,78.24,7.00,,NaN
138,138,2439376,Tahoua,0,NE,1603844023,29,14.89,5.27,77.00,2.24,Bungalows Guesthouse,Tahoua
155,155,2447513,Arlit,0,NE,1603844027,15,18.74,7.39,79.00,7.78,Hôtel Telwa Bungalow,"Route Tahoua Arlit, RN25, Agadez"


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in NarrowDF.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))